<a href="https://colab.research.google.com/github/georgsmeinung/simulacion-en-ciencia-datos/blob/main/Simulacion_Actividad_en_Clase_Emergencia_M%C3%A9dica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Actividad en clase**

* Simulación de atención en una guardia médica (M/M/c + prioridades)
* Objetivo: Modelar un servicio de emergencia hospitalaria con diferentes niveles de urgencia (rojo, amarillo, verde). Aspectos a analizar:
    * Tasa de llegadas por clase (Poisson)
    * Tiempos de servicio exponenciales distintos
    * Prioridad no-preemptiva
    * Cálculo de métricas por clase (espera, ocupación, tiempo total)
* Preguntas guía:
    * ¿Cuál es el tiempo promedio de espera por nivel de urgencia?
    * ¿Qué porcentaje de pacientes críticos esperan más de cierto umbral?
    * ¿Qué número de médicos garantiza un SLA de 90% en < 10 minutos para pacientes rojos?
* Salida esperada: tabla y gráficos comparativos de tiempos de espera, heatmap por clase.
* Prompt sugerido para ChatGPT:”Usando el modelo de estación de servicio adaptado al contexto hospitalario, simulá una guardia médica con 3 clases de pacientes y prioridad no-preemptiva. Mostrá los KPIs por clase y graficá la evolución de la cola.”

In [2]:
!pip install simpy

In [3]:
import simpy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Parámetros de Simulación ---

RANDOM_SEED = 42             # Semilla para reproducibilidad
NUM_DOCTORES = 3             # Número de médicos (el 'c' en M/M/c)
SIM_TIME = 1440              # Tiempo total de simulación (ej. 1440 min = 24 horas)

# Prioridades (0 es la más alta en simpy)
PRIORIDADES = {"Rojo": 0, "Amarillo": 1, "Verde": 2}
CLASES_POR_PRIORIDAD = {v: k for k, v in PRIORIDADES.items()}

# Tiempos promedio de LLEGADA (en minutos) - Inverso de la tasa (1/lambda)
# Es el tiempo *entre* llegadas de cada clase
TIEMPO_LLEGADA = {
    "Rojo": 60.0,      # Un paciente rojo llega cada 60 min en promedio
    "Amarillo": 30.0,    # Un paciente amarillo llega cada 30 min en promedio
    "Verde": 20.0        # Un paciente verde llega cada 20 min en promedio
}

# Tiempos promedio de SERVICIO (en minutos) - (1/mu)
TIEMPO_SERVICIO = {
    "Rojo": 25.0,      # El servicio a un paciente rojo dura 25 min en promedio
    "Amarillo": 15.0,    # El servicio a un paciente amarillo dura 15 min en promedio
    "Verde": 10.0       # El servicio a un paciente verde dura 10 min en promedio
}

# Lista para guardar todas las métricas de los pacientes
metricas_pacientes = []

In [4]:
# --- Procesos de Simulación ---

def paciente(env, id_paciente, clase, medicos):
    """Proceso que modela el flujo de un paciente."""
    t_llegada = env.now

    # Obtener la prioridad numérica para simpy
    prioridad = PRIORIDADES[clase]

    # Pedir un médico con la prioridad asignada
    # simpy.PriorityResource maneja la cola no-preemptiva automáticamente
    with medicos.request(priority=prioridad) as request:
        yield request

        # --- Inicio de la atención ---
        t_inicio_servicio = env.now
        t_espera = t_inicio_servicio - t_llegada

        # Simular tiempo de servicio (distribución exponencial)
        t_servicio = random.expovariate(1.0 / TIEMPO_SERVICIO[clase])
        yield env.timeout(t_servicio)

        # --- Fin de la atención ---
        t_salida = env.now
        t_total_sistema = t_salida - t_llegada

        # Guardar métricas
        metricas_pacientes.append({
            "id": id_paciente,
            "clase": clase,
            "t_llegada": t_llegada,
            "t_inicio_servicio": t_inicio_servicio,
            "t_espera": t_espera,
            "t_servicio": t_servicio,
            "t_salida": t_salida,
            "t_total_sistema": t_total_sistema
        })

def generador_por_clase(env, medicos, clase, id_offset):
    """Genera pacientes de una clase específica."""
    id_paciente = id_offset
    while True:
        # Tiempo entre llegadas (distribución exponencial para un proceso Poisson)
        t_entre_llegada = random.expovariate(1.0 / TIEMPO_LLEGADA[clase])
        yield env.timeout(t_entre_llegada)

        id_paciente += 1
        # Crear el proceso del nuevo paciente
        print(f"t={env.now:.2f}: Llega paciente {id_paciente} ({clase})")
        env.process(paciente(env, id_paciente, clase, medicos))

def generador_principal(env, medicos):
    """Inicia los generadores para cada clase de paciente."""
    # Iniciar un generador separado para cada clase
    # Cada uno corre en paralelo con su propia tasa de llegada
    env.process(generador_por_clase(env, medicos, "Rojo", 0))
    env.process(generador_por_clase(env, medicos, "Amarillo", 10000))
    env.process(generador_por_clase(env, medicos, "Verde", 20000))

In [6]:
# --- Ejecución de la Simulación ---

print(f"Iniciando simulación de Guardia Médica (M/M/c con prioridades)")
print(f"Doctores: {NUM_DOCTORES}, Tiempo Sim.: {SIM_TIME} min (24 horas)")
print("-" * 40)

random.seed(RANDOM_SEED)

# Crear el entorno de simulación
env = simpy.Environment()

# Crear el recurso (médicos)
# Usamos PriorityResource para manejar la prioridad no-preemptiva
medicos = simpy.PriorityResource(env, capacity=NUM_DOCTORES)

# Iniciar el generador de pacientes
env.process(generador_principal(env, medicos))

# Correr la simulación
env.run(until=SIM_TIME)

print("-" * 40)
print("Simulación terminada.")

Iniciando simulación de Guardia Médica (M/M/c con prioridades)
Doctores: 3, Tiempo Sim.: 1440 min (24 horas)
----------------------------------------


ValueError: None is not a generator.

In [ ]:
# --- Análisis y Salida de Resultados ---

if not metricas_pacientes:
    print("No se generaron métricas. La simulación pudo ser muy corta.")
else:
    # 1. Convertir a DataFrame de Pandas para fácil análisis
    df_metricas = pd.DataFrame(metricas_pacientes)

    # 2. Calcular KPIs (Respondiendo a "Cálculo de métricas por clase")
    kpis = df_metricas.groupby('clase').agg(
        pacientes_atendidos=('id', 'count'),
        t_espera_promedio=('t_espera', 'mean'),
        t_espera_max=('t_espera', 'max'),
        t_sistema_promedio=('t_total_sistema', 'mean'),
        t_servicio_promedio=('t_servicio', 'mean')
    ).reindex(PRIORIDADES.keys()) # Ordenar por prioridad

    print("\n--- KPIs por Clase de Paciente ---")
    print(kpis.to_markdown(floatfmt=".2f"))

    # 3. Responder Preguntas Guía (parcialmente)

    # "¿Qué porcentaje de pacientes críticos esperan más de cierto umbral?"
    UMBRAL_ROJO = 10 # 10 minutos
    pacientes_rojos = df_metricas[df_metricas['clase'] == 'Rojo']
    if not pacientes_rojos.empty:
        pct_espera_critica = (pacientes_rojos['t_espera'] > UMBRAL_ROJO).mean() * 100
        print(f"\n--- Análisis Pacientes Rojos (Críticos) ---")
        print(f"Porcentaje que espera > {UMBRAL_ROJO} min: {pct_espera_critica:.2f}%")

        # "¿Qué número de médicos garantiza un SLA de 90% en < 10 minutos para pacientes rojos?"
        sla_percentil_90 = pacientes_rojos['t_espera'].quantile(0.90)
        sla_cumplido = (pacientes_rojos['t_espera'] < UMBRAL_ROJO).mean() >= 0.90

        print(f"Percentil 90 de espera: {sla_percentil_90:.2f} min")
        print(f"SLA (90% atendido en < {UMBRAL_ROJO} min): {'CUMPLIDO' if sla_cumplido else 'NO CUMPLIDO'}")

    # 4. Generar Gráficos (Salida esperada: "gráficos comparativos de tiempos de espera")

    # --- Gráfico 1: Tiempos de Espera por Clase (Boxplot) ---
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df_metricas, x='clase', y='t_espera', order=PRIORIDADES.keys())
    plt.title('Distribución del Tiempo de Espera por Clase')
    plt.xlabel('Clase de Paciente')
    plt.ylabel('Tiempo de Espera (min)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # --- Gráfico 2: "Heatmap por clase" (interpretado como tiempos de espera vs. llegada) ---
    # Un scatter plot con colores por clase es más claro que un heatmap aquí.
    plt.figure(figsize=(12, 7))
    sns.scatterplot(data=df_metricas, x='t_llegada', y='t_espera', hue='clase',
                    hue_order=PRIORIDADES.keys(), style='clase', s=50, alpha=0.7)
    plt.title('Tiempo de Espera vs. Tiempo de Llegada')
    plt.xlabel('Hora de Llegada (minutos desde inicio)')
    plt.ylabel('Tiempo de Espera (min)')
    plt.legend(title='Clase')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

**Actividad en clase**

* Optimización de la estación de servicio con mezcla de vehículos
* Objetivo: Experimentar con configuraciones de surtidores (número y tipo) para minimizar la espera media sin sobredimensionar.
* Aspectos:
    * Llegadas no homogéneas (pico matutino)
    * Distribuciones lognormales de servicio
    * Capacidad finita de cola
* Preguntas guía:
    * ¿Qué configuración de surtidores logra una ocupación < 90% y tiempo medio < 3 minutos?
    * ¿Cómo impacta la llegada de camiones (servicio más largo)?
    * ¿Qué ocurre si se agrega una "pista rápida" solo para motos?
* Salida esperada: curva de trade-off espera vs. capacidad y ocupación.
* Prompt sugerido: "A partir del código de simulación de la estación de servicio, realizá un experimento de sensibilidad variando el número de surtidores de 4 a 10 y graficá el tiempo promedio de espera y la utilización."